In [29]:
import os
import json
from tqdm.notebook import tqdm
from pytube import Channel

In [30]:
GREAT_SCOTT = "https://www.youtube.com/channel/UC6mIxFTvXkWQVEHPsEdflzQ"

In [31]:
def get_videos(name, channel_url):
    # check to see if we've cached this channel already
    if os.path.exists(name+".json"):
        # if we have, load the cache
        with open(name, "r") as f:
            videos = json.load(f)
            return videos
    channel = Channel(channel_url)
    videos = channel.videos
    print("Extracting data from "+str(len(videos))+" videos")
    video_data = [{
        "title": video.title,
        "thumbnail": video.thumbnail_url,
        "description": video.description,
        "publish_date": video.publish_date,
        "views": video.views,
        "length": video.length
    } for video in tqdm(videos)]
    # cache the results
    with open(name+".json", "w") as f:
        json.dump(video_data, f)
    return video_data

In [32]:
print("Downloading videos - this may take some time for a large channel")
videos = get_videos("great_scott", GREAT_SCOTT)

Extracting data from 384 videos


  0%|          | 0/384 [00:00<?, ?it/s]